# Reducing Memory Size for IEEE

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
gc.enable()
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
%%time
train_iden = pd.read_csv('ieee-fraud-detection/train_identity.csv', index_col = 'TransactionID')
train_tran = pd.read_csv('ieee-fraud-detection/train_transaction.csv', index_col = 'TransactionID')
test_iden = pd.read_csv('ieee-fraud-detection/test_identity.csv', index_col = 'TransactionID')
test_tran = pd.read_csv('ieee-fraud-detection/test_transaction.csv', index_col = 'TransactionID')
print('데이터 로드 완료!')

데이터 로드 완료!
Wall time: 42.2 s


In [4]:
print('Shape of Data:')
print(train_tran.shape)
print(test_tran.shape)
print(train_iden.shape)
print(test_iden.shape)

Shape of Data:
(590540, 393)
(506691, 392)
(144233, 40)
(141907, 40)


In [6]:
train = train_tran.merge(train_iden, how='left', left_index=True, right_index=True)
test = test_tran.merge(test_iden, how='left', left_index=True, right_index=True)

In [7]:
del train_tran, train_iden, test_tran, test_iden

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 2987000 to 3577539
Columns: 433 entries, isFraud to DeviceInfo
dtypes: float64(399), int64(3), object(31)
memory usage: 1.9+ GB


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 3663549 to 4170239
Columns: 432 entries, TransactionDT to DeviceInfo
dtypes: float64(399), int64(2), object(31)
memory usage: 1.7+ GB


### Numerical columns
pandas는 df로 읽을 때
- int32, int64, float32, float64

로 읽는다. 우리가 column의 max, min 값을 알고 있다면 더 적은 메모리를 사용하는 type으로 지정하여 사용할 수 있다.

**아래가 우리가 사용할 수 있는 types**
- int8 / uint8 : consumes 1 byte of memory, range between -128/127 or 0/255
- bool : consumes 1 byte, true or false
- float16 / int16 / uint16: consumes 2 bytes of memory, range between -32768 and 32767 or 0/65535
- float32 / int32 / uint32 : consumes 4 bytes of memory, range between -2147483648 and 2147483647
- float64 / int64 / uint64 / datetime64 : consumes 8 bytes of memory

In [1]:
import numpy as np
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------



### Categorical columns
pandas는 categorical column들은 
- object

로 읽는다. *low cardinality*의 column의 경우, **category datatype**을 이용하여 각 value들을 pointer가 아닌 integer로 사용하면 좋다.


### Selecting Types While Reading the Data In
아래의 방법은 data를 읽은 후에 memory reducing 하는 방법이지만 각 column에 대한 정보가 있는 
상태에서 data를 읽으면서 동시에 memory reducing을 하는 것도 가능하다. 

- 이런 식으로!

data_types= {‘col1’: ‘int8’, ‘col2’: ‘float32’}

df = pd.read_csv(data_path, dtype=data_types}

## Reducing Memory Size

In [23]:
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2
    print("해당 df의 메모리 사용량은 : {} MB".format(start_mem_usg))
    NAlist = [] # missing value가 있는 column을 넣습니다
    for col in df.columns:
        if df[col].dtype != object:
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()

            # Integer는 NA가 있으면 안되므로 filled 해준다
            if not np.isfinite(df[col]).all(): # NA값이 있으면
                NAlist.append(col)
                df[col].fillna(mn-1, inplace = True)
                
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True
            if IsInt : 
                if mn >= 0 :
                    if mx < 255 :
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535 :
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295 : 
                        df[col] = df[col].astype(np.uint32)
                    else : 
                        df[col] = df[col].astype(np.uint64)
                else : 
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("최종 해당 df의 메모리 사용량은 : {} MB".format(mem_usg))
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [24]:
train, NAlist = reduce_mem_usage(train)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

해당 df의 메모리 사용량은 : 1975.3707885742188 MB
___MEMORY USAGE AFTER COMPLETION:___
최종 해당 df의 메모리 사용량은 : 748.7585830688477 MB
This is  37.904710720628096 % of the initial size
_________________

_________________

['card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92'

In [25]:
test, NAlist = reduce_mem_usage(test)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

해당 df의 메모리 사용량은 : 1693.867820739746 MB
___MEMORY USAGE AFTER COMPLETION:___
최종 해당 df의 메모리 사용량은 : 635.6199779510498 MB
This is  37.52476847180802 % of the initial size
_________________

_________________

['card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 2987000 to 3577539
Columns: 433 entries, isFraud to DeviceInfo
dtypes: float64(80), int16(7), int8(9), object(31), uint16(63), uint32(2), uint8(241)
memory usage: 748.8+ MB


In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 3663549 to 4170239
Columns: 432 entries, TransactionDT to DeviceInfo
dtypes: float64(80), int16(1), int8(9), object(31), uint16(50), uint32(2), uint8(259)
memory usage: 635.6+ MB


## 주의
- int로 바꾸려다보니 NA값을 임의로 채워야했다 -> 따라서 NA값 처리 후에 하는게 좋지 않을까 싶다.
- 아니면 해당하는 column에 적절한 type을 찾기만하고 read_csv 할 때, 지정하는것도 좋을 것 같다.

마지막으로 이렇게 저장해서 사용할 수 있다.

In [ ]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)